In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm_notebook

In [ ]:
SessionLink = 'https://ipp-ean20.netkey.at/index.php?p=elasearch&t=browsesessions&pnr='
TotalPages = 10

In [ ]:
i = 1
url = SessionLink + str(i)

In [ ]:
response = requests.get(url)
data = BeautifulSoup(response.text,'lxml')

In [ ]:
results = data.find('div',class_ = 'ipp-result-list gallery')
SessionList = results.find_all('div',class_ = 'ipp-result-list-item-info-title')

In [ ]:
SessionLinks = []
for session in SessionList:
    SessionLinks.append(session.find('a')['href'])

In [ ]:
len(SessionLinks)

In [ ]:
SessionLinks = []
numberSessions = 0
for i in range(1,TotalPages+1):
    url = SessionLink + str(i)
    response = requests.get(url)
    data = BeautifulSoup(response.text,'lxml')
    results = data.find('div',class_ = 'ipp-result-list gallery')
    SessionList = results.find_all('div',class_ = 'ipp-result-list-item-info-title')
    for session in SessionList:
        SessionLinks.append(session.find('a')['href'])
    numberSessions = len(SessionLinks) - numberSessions
    print(len(SessionList))
    

In [ ]:
sessUrl = 'https://ipp-ean20.netkey.at/index.php?p=recorddetail&rid=be19e9fb-d280-48f3-874f-f62b2bba747f&t=browsesessions'
sessionResp = requests.get(sessUrl)
sessionInfo = BeautifulSoup(sessionResp.text,'lxml')

In [ ]:
SessionDetails = sessionInfo.find('div',class_ = 'ipp-record-detail-infos')
SessionDetails

In [ ]:
SessionDict = {}
SessionDict['Session Heading'] = SessionDetails.find('div',class_ = 'ipp-record-detail-title').text.strip()
SessionDict['Session Time'] = SessionDetails.find('span',class_ = 'ipp-record-detail-datetime').text.replace(' ','').strip()
try:
    SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[0].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[0].text.strip()
except:
    pass
try:
    SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[1].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[1].text.strip()
except:
    pass
try:
    SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[2].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[2].text.strip()
except:
    pass
try:
    SessionDict['Session Description'] = sessionInfo.find('div',class_ = 'ipp-record-detail-description').text
except:
    pass
print(SessionDict)

In [ ]:
PresentationDetails = sessionInfo.find('div',class_ = 'ipp-record-detail-children')
try:
    Presentations = PresentationDetails.find_all('div',class_ = 'ipp-record-detail ipp-record-detail-child')
    print(Presentations)
except:
    Presentations = []

In [ ]:
try:
    PresentationCode = Presentations[0].find('div',class_ = 'ipp-record-detail-child-number').text.strip()
except:
    PresentationCode = ''
try:
    PresentationTitle = Presentations[0].find('div',class_ = 'ipp-record-detail-title').text.strip()
except:
    PresentationTitle = ''
try:    
    PresentationAuthors = Presentations[0].find('div',class_ = 'ipp-record-detail-author-block').text.strip()
except:
    PresentationAuthors = ''
try:
    PresentationAbstract = Presentations[0].find('div',class_ = 'ipp-record-detail-abstract-content').text.strip()
except:
    PresentationAbstract = ''
print(PresentationCode)
print(PresentationTitle)
print(PresentationAuthors)
print(PresentationAbstract)

In [1]:
Masterdf = pd.DataFrame()

for i in tqdm_notebook(SessionLinks):
    sessUrl = i
    sessionResp = requests.get(sessUrl)
    sessionInfo = BeautifulSoup(sessionResp.text,'lxml')
    SessionDetails = sessionInfo.find('div',class_ = 'ipp-record-detail-infos')
    SessionDict = {}
    SessionDict['Session Heading'] = SessionDetails.find('div',class_ = 'ipp-record-detail-title').text.strip()
    SessionDict['Session Time'] = SessionDetails.find('span',class_ = 'ipp-record-detail-datetime').text.replace(' ','').strip()
    SessionDict['Url'] = sessUrl
    try:
        SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[0].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[0].text.strip()
    except:
        pass
    try:
        SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[1].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[1].text.strip()
    except:
        pass
    try:
        SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[2].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[2].text.strip()
    except:
        pass
    try:
        SessionDict['Session Description'] = sessionInfo.find('div',class_ = 'ipp-record-detail-description').text
    except:
        pass
    PresentationDetails = sessionInfo.find('div',class_ = 'ipp-record-detail-children')
    try:
        Presentations = PresentationDetails.find_all('div',class_ = 'ipp-record-detail ipp-record-detail-child')
#         print(Presentations)
    except:
        Presentations = []
#     presentationDict = SessionDict.copy()
    if(len(Presentations)==0):
        print(i)
        Masterdf = Masterdf.append(SessionDict,ignore_index = True)
    else:
        for i in tqdm_notebook(range(len(Presentations))):
            presentationDict = SessionDict.copy()
            try:
                PresentationCode = Presentations[0].find('div',class_ = 'ipp-record-detail-child-number').text.strip()
            except:
                PresentationCode = ''
            try:
                PresentationTitle = Presentations[0].find('div',class_ = 'ipp-record-detail-title').text.strip()
            except:
                PresentationTitle = ''
            try:    
                PresentationAuthors = Presentations[0].find('div',class_ = 'ipp-record-detail-author-block').text.strip()
            except:
                PresentationAuthors = ''
            try:
                PresentationAbstract = Presentations[0].find('div',class_ = 'ipp-record-detail-abstract-content').text.strip()
            except:
                PresentationAbstract = ''
            presentationDict['Presentation Code'] = PresentationCode
            presentationDict['Presentation Title'] = PresentationTitle
            presentationDict['Presentation Authors'] = PresentationAuthors
            presentationDict['Presentation Abstract'] = PresentationAbstract
            Masterdf.append(presentationDict,ignore_index = True)
Masterdf    



NameError: name 'pd' is not defined